# Data cleaning

This notebooks is for the step by step data cleaning process. All datasets that will be used and need data cleaning, will be processed in this file and converted to a csv file so that it can be easily used in the project

In [3]:
import pandas as pd
import numpy as np
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import os
import re
from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")

## EmotioNL

EmotioNL dataset is a dutch emotion dataset with dutch sentences labeled with six emotions (joy, sadness, fear, neutral, love and anger). The dataset is split in two parts. The parts are:

1. A dataset of 1000 tweets
2. A dataset of 1000 sentences from different tv shows

Both parts are stored in textfiles and contain unnecessary columns for this project. We filter the files so that only the correct columns remain and convert both to a csv file.

### EmotioNL TV shows

Get the path to the file

In [4]:
base_path = "datasets\EmotioNL_dataset_open"
filename = "EmotioNL_captions.txt"

pf = os.path.join(base_path, filename)
pf

'datasets\\EmotioNL_dataset_open\\EmotioNL_captions.txt'

The current dataset. Consisting of 7 columns.

In [28]:
shows_df = pd.read_csv(pf, sep="\t")
shows_df

,ID,Text,Show,Valence,Arousal,Dominance,Category
0,0,Kweenie of da gij al es goed hebt stilgestaan ...,OV,0.303732,0.513015,0.664205,neutral
1,1,Santé.,BG,0.712377,0.492071,0.619150,joy
2,10,Ja jongens. Dees is nie gezond ze. Pfff. Dees ...,BG,0.338710,0.827688,0.260467,fear
3,100,Ik had de vorige keer ook al zoiets van: nie m...,OV,0.438437,0.714087,0.471438,anger
4,101,Tis wat het is. Tis ook een belangrijke job eh...,BZL,0.696067,0.552524,0.705887,neutral
...,...,...,...,...,...,...,...
995,995,"Jah ze werken harder eh als wij in België, en ...",BZL,0.344995,0.849752,0.543079,sadness
996,996,Ah gij. Wa een ramp zijt gij. Allemaal hetzelfde.,OV,0.257216,0.690051,0.462198,anger
997,997,"Der sta Lego van Lord of the Rings, van Star W...",BG,0.681752,0.434160,0.673334,neutral
998,998,Nu zijn we precies aant kamperen.,BZL,0.503272,0.293365,0.529894,joy


There are no empty values nor duplicates in this dataset

In [29]:
dupes = shows_df[shows_df.duplicated()]
empty = np.where(pd.isnull(shows_df))

len(dupes), len(empty[0])

(0, 0)

 For this project, we only need the columns text and category.

In [30]:
new_s_df = shows_df.drop(["ID", "Show", "Valence", "Arousal", "Dominance"], axis=1)
new_s_df

,Text,Category
0,Kweenie of da gij al es goed hebt stilgestaan ...,neutral
1,Santé.,joy
2,Ja jongens. Dees is nie gezond ze. Pfff. Dees ...,fear
3,Ik had de vorige keer ook al zoiets van: nie m...,anger
4,Tis wat het is. Tis ook een belangrijke job eh...,neutral
...,...,...
995,"Jah ze werken harder eh als wij in België, en ...",sadness
996,Ah gij. Wa een ramp zijt gij. Allemaal hetzelfde.,anger
997,"Der sta Lego van Lord of the Rings, van Star W...",neutral
998,Nu zijn we precies aant kamperen.,joy


Now we convert the new dataframe to a useable csv file to train our models with

In [31]:
new_s_df.to_csv("datasets\emotionl_shows.csv", index=False, mode="w+")

PermissionError: [Errno 13] Permission denied: 'datasets\\emotionl_shows.csv'

In [32]:
pd.read_csv("datasets\emotionl_shows.csv")

,Text,Category
0,Kweenie of da gij al es goed hebt stilgestaan ...,neutral
1,Santé.,joy
2,Ja jongens. Dees is nie gezond ze. Pfff. Dees ...,fear
3,Ik had de vorige keer ook al zoiets van: nie m...,anger
4,Tis wat het is. Tis ook een belangrijke job eh...,neutral
...,...,...
995,"Jah ze werken harder eh als wij in België, en ...",sadness
996,Ah gij. Wa een ramp zijt gij. Allemaal hetzelfde.,anger
997,"Der sta Lego van Lord of the Rings, van Star W...",neutral
998,Nu zijn we precies aant kamperen.,joy


### EmotioNL Tweets

Get the path file

In [5]:
filename = "EmotioNL_tweets.txt"

pf = os.path.join(base_path, filename)
pf

'datasets\\EmotioNL_dataset_open\\EmotioNL_tweets.txt'

The dataset consists of 6 columns.

In [6]:
tweets_df = pd.read_csv(pf, sep="\t")
tweets_df

,ID,Tweet ID,Valence,Arousal,Dominance,Category
0,4,['914793795094446082'],0.553364,0.550661,0.732408,anger
1,7,['902122434714841090'],0.074637,0.754539,0.732349,anger
2,8,['864400774251589632'],0.395079,0.838521,0.768225,anger
3,11,['894957660889591810'],0.069375,0.626347,0.647728,anger
4,15,['905476032320462848'],0.150416,0.697961,0.584830,anger
...,...,...,...,...,...,...
995,949,['848596718740545536'],0.409241,0.635159,0.132824,sadness
996,951,['912331156317540358'],0.174462,0.964262,0.349197,sadness
997,963,['913090078582431744'],0.171635,0.781369,0.226771,sadness
998,976,['834451051407360000'],0.078343,0.422965,0.071597,sadness


The dataset has no empty or duplicate values

In [7]:
dupes = tweets_df[tweets_df.duplicated()]
empty = np.where(pd.isnull(tweets_df))

len(dupes), len(empty[0])

(0, 0)

We need only need the tweet ID and the category

In [8]:
new_t_df = tweets_df.drop(["ID", "Valence", "Arousal", "Dominance"], axis=1)
new_t_df

,Tweet ID,Category
0,['914793795094446082'],anger
1,['902122434714841090'],anger
2,['864400774251589632'],anger
3,['894957660889591810'],anger
4,['905476032320462848'],anger
...,...,...
995,['848596718740545536'],sadness
996,['912331156317540358'],sadness
997,['913090078582431744'],sadness
998,['834451051407360000'],sadness


In [9]:
new_t_df.insert(1, "text", '')
new_t_df

,Tweet ID,text,Category
0,['914793795094446082'],,anger
1,['902122434714841090'],,anger
2,['864400774251589632'],,anger
3,['894957660889591810'],,anger
4,['905476032320462848'],,anger
...,...,...,...
995,['848596718740545536'],,sadness
996,['912331156317540358'],,sadness
997,['913090078582431744'],,sadness
998,['834451051407360000'],,sadness


This dataset needs a lot more data processing. We need to get the text from the tweet id's. For this we will use a simple url call using the requests module.

In [10]:
""" make a html session, because jupyter has it's own loop running in the background we use a AsyncHTMLSession.
This session allows us to use the request html session method in a jupyter notebook

"""
asession = AsyncHTMLSession()

A tweet ID in the dataset has symbols and brackets that need to be removed. 

In [11]:
current_id = new_t_df["Tweet ID"][0]
new_id = new_t_df["Tweet ID"][0].strip("[").strip("]").strip("'")
print(current_id, new_id)

['914793795094446082'] 914793795094446082


In [12]:
def clean_string(text):
#     pattern = re.compile("["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002500-\U00002BEF"  # chinese char
#         u"\U00002702-\U000027B0"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U00010000-\U0010ffff"
#         u"\u2640-\u2642" 
#         u"\u2600-\u2B55"
#         u"\u200d"
#         u"\u23cf"
#         u"\u23e9"
#         u"\u231a"
#         u"\ufe0f"  # dingbats
#         u"\u3030"
#                       "]+", re.UNICODE)
#     r_text = pattern.sub(r' ', text) # no emoji
    r_text = re.sub(r'^.*?"', '"', text) # no username
    r_text = r_text.replace(" / X", "") # No " at the end
    r_text = re.sub(r'@.*?(?=\s)', '', r_text) # remove @'s'
    
    return r_text

In [27]:
url = "https://twitter.com/anyuser/status/885071203140071424"
print(url)
r = await asession.get(url)
await r.html.arender(sleep=2)
# fetch the raw html code
resp=r.html.raw_html
# process it using bs4
soup = BeautifulSoup(resp, "lxml")
soup.prettify()
# print(soup.prettify())
# get the correct data and append
text = soup.title.text
print(text)
print(tokenizer(text))
text = clean_string(text)
print(text)
print(tokenizer(text))

https://twitter.com/anyuser/status/885071203140071424
charlottedekens. on Twitter: "@VBBenthe Oei😕als er iets is stuur maar ❤️❤️" / X
{'input_ids': [0, 685, 13802, 6308, 98, 4149, 4, 510, 4089, 30, 165, 1009, 7895, 4573, 1019, 15516, 171, 25993, 5475, 1182, 31, 176, 12, 2311, 37, 9163, 5617, 6032, 29166, 8366, 5617, 6032, 29166, 260, 433, 1138, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
" Oei😕als er iets is stuur maar ❤️❤️"
{'input_ids': [0, 260, 15516, 171, 25993, 5475, 1182, 31, 176, 12, 2311, 37, 9163, 5617, 6032, 29166, 8366, 5617, 6032, 29166, 260, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [191]:
# sample of 3 tweets using the request method
for i in range(3):
    id = tweets_df['Tweet ID'][i].strip("[").strip("]").strip("'")
    print(id)
    # call the url
    url = "https://twitter.com/anyuser/status/{}".format(id)
    print(url)
    r = await asession.get(url)
    await r.html.arender(sleep=2)
    # fetch the raw html code
    resp=r.html.raw_html
    # process it using bs4
    soup = BeautifulSoup(resp, "lxml")
    soup.prettify()
    # get the correct data and append
    text = soup.title.text
    n_text = clean_string(text)
    print(n_text)

914793795094446082
https://twitter.com/anyuser/status/914793795094446082
" Maandagmorgen en extra druk Dat is nu voordeel van niet meer werken op maandag al dit nieuws lezen!"
902122434714841090
https://twitter.com/anyuser/status/902122434714841090
" Echt...zweet en shag..en dat een hele dag Je zal ermee op kantoor zitten "
864400774251589632
https://twitter.com/anyuser/status/864400774251589632
"   D66Pechtold zei 1e BELANGRIJKSTE!poging dream on Pechtold gelul id ruimte! PVV id 2e grtste!wat e kleuterklasje.pesten uitsluiten z nie 2017"


In [17]:
texts

['Voeten on Twitter: "@LINDAnieuws Maandagmorgen en extra druk😣Dat is nu voordeel van niet meer werken op maandag al dit nieuws lezen!" / X',
 'Les on Twitter: "@9Owen1 Echt...zweet en shag..en dat een hele dag😡Je zal ermee op kantoor zitten😝" / X',
 'bilblebons on Twitter: "@ohzitdatzo @telegraaf @VVD D66Pechtold zei 1e BELANGRIJKSTE!poging😂dream on Pechtold gelul id ruimte! PVV id 2e grtste!wat e kleuterklasje.pesten uitsluiten z nie 2017" / X',
 'jacky de deugd on Twitter: "@PostNL heeft mijn pakket om 17.02 bezorgd alleen niet bij mij😡leuk vakantiekrachten nu moet ik wachten!waar is mijn pakje" / X',
 'Tweet / X',
 'Tweet / X',
 'Renata Hofman💚\U0001f90d on Twitter: "https://t.co/GTCqMzQXkp Ik moest lachen😂De reden dat ik niks met Maduro had in,1 bericht #Amsterdamsearrogantie#Nikslatenzien #bestemaarweer" / X',
 'Verwarde van Staat, verwarde™ (gek) on Twitter: "@VerwardeMens ZWARTgallig🤔er zijn ook nog andere kleuren hoor.\npaars groen blauw" / X',
 'Patrick 🚴🏻\u200d♂️📸📱💻 on Twitt

In [13]:
i = 0
for index, row in new_t_df.iterrows():
    id = tweets_df['Tweet ID'][i].strip("[").strip("]").strip("'")
    # call the url
    url = "https://twitter.com/anyuser/status/{}".format(id)
    r = await asession.get(url)
    await r.html.arender(sleep=2)
    # fetch the raw html code
    resp=r.html.raw_html
    # process it using bs4
    soup = BeautifulSoup(resp, "lxml")
    soup.prettify()
    # get the correct data and append
    text = soup.title.text
    if text !=  "Post / X":
        clean_text =  clean_string(text)
        new_t_df.loc[index]["text"] = clean_text
    else:
        new_t_df.drop([index])
    i += 1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472


The last step is removing emoji's from the tweets, and remove the twitter names and "/ X".

In [14]:
new_t_df

,Tweet ID,text,Category
0,['914793795094446082'],""" Maandagmorgen en extra druk😣Dat is nu voorde...",anger
1,['902122434714841090'],""" Echt...zweet en shag..en dat een hele dag😡Je...",anger
2,['864400774251589632'],""" D66Pechtold zei 1e BELANGRIJKSTE!poging😂dr...",anger
3,['894957660889591810'],""" heeft mijn pakket om 17.02 bezorgd alleen ni...",anger
4,['905476032320462848'],,anger
...,...,...,...
995,['848596718740545536'],,sadness
996,['912331156317540358'],,sadness
997,['913090078582431744'],,sadness
998,['834451051407360000'],,sadness


In [15]:
new_t_df.loc[new_t_df["text"] == ""]

,Tweet ID,text,Category
4,['905476032320462848'],,anger
5,['850095356780175360'],,anger
11,['902928999109857280'],,anger
12,['918217768025182209'],,anger
13,['915974322593964032'],,anger
...,...,...,...
995,['848596718740545536'],,sadness
996,['912331156317540358'],,sadness
997,['913090078582431744'],,sadness
998,['834451051407360000'],,sadness


In [16]:
final_df = new_t_df.drop(new_t_df[new_t_df['text'] == ""].index)

In [17]:
final_df.head(10)

,Tweet ID,text,Category
0,['914793795094446082'],""" Maandagmorgen en extra druk😣Dat is nu voorde...",anger
1,['902122434714841090'],""" Echt...zweet en shag..en dat een hele dag😡Je...",anger
2,['864400774251589632'],""" D66Pechtold zei 1e BELANGRIJKSTE!poging😂dr...",anger
3,['894957660889591810'],""" heeft mijn pakket om 17.02 bezorgd alleen ni...",anger
6,['862334234706530304'],"""https://t.co/GTCqMzQXkp Ik moest lachen😂De re...",anger
7,['929448896773976064'],""" ZWARTgallig🤔er zijn ook nog andere kleuren h...",anger
8,['931421806636032001'],""" Sterkte😱normaal een thuiswerkdag op vrijdag ...",anger
9,['921705689956913152'],"""Duizenden afgewezen asielzoekers toch in Nede...",anger
10,['826851690905292815'],"""Voor die duizenden😏mensen op het #malieveld o...",anger
14,['898939776258256897'],""" U😂beiden snapt dan niks van 'gevoel'. --...",anger


In [18]:
final_df

,Tweet ID,text,Category
0,['914793795094446082'],""" Maandagmorgen en extra druk😣Dat is nu voorde...",anger
1,['902122434714841090'],""" Echt...zweet en shag..en dat een hele dag😡Je...",anger
2,['864400774251589632'],""" D66Pechtold zei 1e BELANGRIJKSTE!poging😂dr...",anger
3,['894957660889591810'],""" heeft mijn pakket om 17.02 bezorgd alleen ni...",anger
6,['862334234706530304'],"""https://t.co/GTCqMzQXkp Ik moest lachen😂De re...",anger
...,...,...,...
885,['867499142892982274'],"""Ik MOET gelukkig zijn.NU! Nee😉Daar is 't leve...",neutral
886,['895874918822195205'],"""Goedemorgen maak er een mooie dag van je was...",neutral
887,['910627665610264582'],""" Slaap ze😴weltrusten""",neutral
890,['832145988525633536'],"""Haak aan bij het Brei-Ontbijt😃Reserveer info ...",neutral


In [19]:
final_df.to_csv("datasets\emotionl_tweets.csv", index=False, mode="w+")

In [ ]:
for i in range(len(texts)):
    pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]",
        flags=re.UNICODE)
    r_text = pattern.sub(r'', texts[i]) # no emoji
    r_text = re.sub(r'^.*?@', '@', r_text) # no username
    r_text = re.sub(re.escape('"')+'.*',"", r_text) # No " at the end
    r_text = re.sub(r'@.*?(?=\s)', '', r_text) # remove @'s'
    n_text.append(r_text)

In [21]:
pd.read_csv("datasets\emotionl_tweets.csv")

,Tweet ID,text,Category
0,['914793795094446082'],""" Maandagmorgen en extra druk😣Dat is nu voorde...",anger
1,['902122434714841090'],""" Echt...zweet en shag..en dat een hele dag😡Je...",anger
2,['864400774251589632'],""" D66Pechtold zei 1e BELANGRIJKSTE!poging😂dr...",anger
3,['894957660889591810'],""" heeft mijn pakket om 17.02 bezorgd alleen ni...",anger
4,['862334234706530304'],"""https://t.co/GTCqMzQXkp Ik moest lachen😂De re...",anger
...,...,...,...
213,['867499142892982274'],"""Ik MOET gelukkig zijn.NU! Nee😉Daar is 't leve...",neutral
214,['895874918822195205'],"""Goedemorgen maak er een mooie dag van je was...",neutral
215,['910627665610264582'],""" Slaap ze😴weltrusten""",neutral
216,['832145988525633536'],"""Haak aan bij het Brei-Ontbijt😃Reserveer info ...",neutral
